In [1]:
import shorttext
import numpy as np

/Users/stephenhky/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]")
conf = conf.setAppName("NIH data")
sc = SparkContext(conf=conf)

In [67]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, MapType, ArrayType

In [4]:
nihdf = shorttext.data.nihreports(sample_size=None)

In [5]:
preprocess = shorttext.utils.standard_text_preprocessor_1()

In [6]:
preprocess(nihdf['NCI'][0])

'pak signal target breast cancer progress'

In [7]:
nihsc = sc.parallelize(nihdf['NCI'])

In [8]:
sctokenizenihdf = nihsc.map(lambda s: (s, len(s.split(" "))))

In [9]:
sctokenizenihdf.toDebugString()

'(4) PythonRDD[1] at RDD at PythonRDD.scala:52 []\n |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:194 []'

In [11]:
sqlContext = SQLContext(sc)
sctokenizenihdf = sqlContext.createDataFrame(sctokenizenihdf, 
                                             StructType([StructField('col1', StringType(), True), 
                                                         StructField('col2', IntegerType(), True)]))

In [12]:
sctokenizenihdf.cache()

DataFrame[col1: string, col2: int]

In [13]:
sctokenizenihdf.show()

+--------------------+----+
|                col1|col2|
+--------------------+----+
|Pak1 Signaling an...|   8|
|RF-SRC: A Unified...|   5|
|NCIC Clinical Tri...|  10|
|Predicting defici...|  10|
|Gold Nanoparticle...|  10|
|Targeting of Myc-...|   9|
|The role of ETV6 ...|   9|
|Growth Factor Ima...|   3|
|Factors for Epige...|   8|
|Identification of...|  10|
|Developing an Acc...|  11|
|Lung Cancer Dispa...|   9|
|Targeting Ceramid...|   5|
|Mechanism of acti...|   9|
|Identifying Cell ...|  11|
|Negative regulati...|  10|
|Core B: Product C...|   8|
|Cellular molecula...|   7|
|Generalized wavef...|   7|
|ATP-Dependent Chr...|   6|
+--------------------+----+
only showing top 20 rows



In [23]:
sctokenizenihdf.write.save('counting')

In [15]:
sctokenizenihdf.registerTempTable("wordcount_table")

In [16]:
sqlgt10 = sqlContext.sql("select * from wordcount_table where col2>=10")

In [19]:
for item in sqlgt10.collect():
    print(item)

Row(col1=u'NCIC Clinical Trials Group - Canadian Collaborating Clinical Trials Network', col2=10)
Row(col1=u'Predicting deficits in social competence for pediatric brain tumor survivors', col2=10)
Row(col1=u'Gold Nanoparticle Assisted Delivery of Cancer Vaccines and Immune Stimulants', col2=10)
Row(col1=u'Identification of LSD1 inhibitors targeting epigenetic regulation in tumor cells', col2=10)
Row(col1=u'Developing an Accurate &Reliable Method for Tumor Motion Monitoring Using (Wei)', col2=11)
Row(col1=u'Identifying Cell Stress Proteins that Predict Clinical Response in Pediatric AML', col2=11)
Row(col1=u'Negative regulation by proteasomal degradation of receptor tyrosine kinase ErbB3', col2=10)
Row(col1=u'The Dana-Farber Cancer Institute Cancer Target Discovery and Development Center', col2=10)
Row(col1=u'Improving Outcomes by Optimally Exploiting Physical and Biological Characteristics of Protons', col2=11)
Row(col1=u'Randomized Phase II study of borteozmibEPOCH-R in Mantle cell ly

In [24]:
subdict = shorttext.data.subjectkeywords()

In [25]:
scsubdict = sc.parallelize(subdict, )

In [26]:
scsubdict

ParallelCollectionRDD[32] at parallelize at PythonRDD.scala:194

In [43]:
scsubdict.saveAsPickleFile('subkeyword_pickle')

In [32]:
scsubdict.count()

3

In [35]:
scsubdict.map(len).collect()

[11, 7, 8]

In [51]:
import pickle, json
scpkl_subdict = sc.parallelize(json.dumps(subdict))

In [53]:
scpkl_subdict.saveAsTextFile('subkeyword_dict')

In [54]:
scpkl_subdict1 = sc.textFile('subkeyword_dict')

In [61]:
scpkl_subdict1

subkeyword_dict MapPartitionsRDD[66] at textFile at NativeMethodAccessorImpl.java:0

In [68]:
scsubdf = sqlContext.createDataFrame(scsubdict, StructType([StructField('dictionary', MapType(StringType(), ArrayType(StringType())), True)]))

In [72]:
scsubdf.write.json('subkeyword_json')

Py4JJavaError: An error occurred while calling o415.json.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc